In [26]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

file_path = r"C:\Users\spasumarthi\Desktop\IBM Data Science\automobile_data.csv"
df = pd.read_csv(file_path) 
# Task 1.1 – Line Plot of Sales Over Years
sales_year = df.groupby('Year')['Sales'].sum()
sales_year.plot(title='Automobile Sales Over Years')
plt.xlabel('Year')
plt.ylabel('Sales')
plt.savefig('Line_plot_1.png')
plt.clf()

# Task 1.2 – Line Plot by Vehicle Type (with Recession Highlight)
for vt in df['Vehicle_Type'].unique():
    subset = df[df['Vehicle_Type'] == vt].sort_values('Year')
    plt.plot(subset['Year'], subset['Sales'], label=vt)
plt.legend()
plt.title("Vehicle Sales Trends by Type")
plt.xlabel("Year")
plt.ylabel("Sales")
plt.savefig("Line_plot_2.png")
plt.clf()

# Task 1.3 – Seaborn Plot for Recession vs Non-Recession
sns.lineplot(data=df, x="Year", y="Sales", hue="Vehicle_Type", style="Recession")
plt.title("Sales Trend by Vehicle Type: Recession vs Non-Recession")
plt.savefig("Bar_Chart.png")
plt.clf()

# Task 1.4 – GDP Subplots for Recession and Non-Recession
fig, axs = plt.subplots(1, 2, figsize=(12, 6))
recession = df[df['Recession'] == True]
non_recession = df[df['Recession'] == False]

axs[0].plot(recession['Year'], recession['GDP'])
axs[0].set_title("GDP During Recession")
axs[1].plot(non_recession['Year'], non_recession['GDP'])
axs[1].set_title("GDP During Non-Recession")

for ax in axs:
    ax.set_xlabel("Year")
    ax.set_ylabel("GDP")

plt.tight_layout()
plt.savefig("Subplot.png")
plt.clf()

# Task 1.5 – Bubble Plot for Seasonality
seasonal = df.groupby(['Season', 'Month'])['Sales'].sum().reset_index()
plt.scatter(seasonal['Month'], seasonal['Sales'], s=seasonal['Sales'] / 500, alpha=0.6)
plt.title("Seasonality Impact on Sales")
plt.xlabel("Month")
plt.ylabel("Sales")
plt.savefig("Bubble.png")
plt.clf()

# Task 1.6 – Scatter Plot: Price vs Sales During Recession
recession_data = df[df['Recession'] == True]
plt.scatter(recession_data['Average_Price'], recession_data['Sales'], alpha=0.6)
plt.title("Price vs Sales During Recession")
plt.xlabel("Average Price")
plt.ylabel("Sales")
plt.savefig("Scatter.png")
plt.clf()

# Task 1.7 – Pie Chart: Ad Spend (Recession vs Non-Recession)
ad_spend = df.groupby('Recession')['Ad_Expenditure'].sum()
labels = ['Recession' if i else 'Non-Recession' for i in ad_spend.index]
plt.pie(ad_spend, labels=labels, autopct='%1.1f%%')
plt.title("Ad Spend: Recession vs Non-Recession")
plt.savefig("Pie_1.png")
plt.clf()

# Task 1.8 – Pie Chart: Ad Spend by Vehicle Type During Recession
recession_ads = df[df['Recession'] == True]
ads_by_type = recession_ads.groupby('Vehicle_Type')['Ad_Expenditure'].sum()
plt.pie(ads_by_type, labels=ads_by_type.index, autopct='%1.1f%%')
plt.title("Ad Spend per Vehicle Type (Recession)")
plt.savefig("Pie_2.png")
plt.clf()

# Task 1.9 – Line Plot: Unemployment vs Sales
for vt in df['Vehicle_Type'].unique():
    vt_data = df[(df['Vehicle_Type'] == vt) & (df['Recession'] == True)]
    plt.plot(vt_data['Unemployment_Rate'], vt_data['Sales'], label=vt)
plt.xlabel("Unemployment Rate")
plt.ylabel("Sales")
plt.title("Unemployment Rate vs Sales During Recession")
plt.legend()
plt.savefig("Line_plot_3.png")
plt.clf()

<Figure size 640x480 with 0 Axes>

<Figure size 1200x600 with 0 Axes>

In [27]:
import pandas as pd
import plotly.express as px
from dash import Dash, dcc, html, Input, Output

# -----------------------------
# TASK 2.1: Create Dash App with Title
# -----------------------------
app = Dash(__name__)
app.title = "Automobile Sales Analysis Dashboard"

# Load data
file_path = r"C:\Users\spasumarthi\Desktop\IBM Data Science\automobile_data.csv"
df = pd.read_csv(file_path)

# -----------------------------
# TASK 2.2: Add Dropdowns
# -----------------------------
vehicle_dropdown = dcc.Dropdown(
    id='vehicle-dropdown',
    options=[{'label': vt, 'value': vt} for vt in df['Vehicle_Type'].unique()],
    value=df['Vehicle_Type'].unique()[0],
    placeholder="Select Vehicle Type"
)

year_dropdown = dcc.Dropdown(
    id='year-dropdown',
    options=[{'label': year, 'value': year} for year in sorted(df['Year'].unique())],
    value=df['Year'].min(),
    placeholder="Select Year"
)

# -----------------------------
# Layout
# -----------------------------
app.layout = html.Div([
    html.H1("Automobile Sales Dashboard", style={"textAlign": "center"}),

    html.Div([
        html.Label("Select Vehicle Type:"),
        vehicle_dropdown,

        html.Br(),

        html.Label("Select Year:"),
        year_dropdown,
    ], style={"width": "40%", "margin": "auto"}),

    html.Br(),

    # -----------------------------
    # TASK 2.3: Output Division
    # -----------------------------
    html.Div(id='output-container',
    className='output-div',
    style={
        "textAlign": "center",
        "fontWeight": "bold",
        "backgroundColor": "#f0f8ff",   # Light blue for contrast
        "padding": "15px",
        "margin": "20px auto",
        "width": "80%",
        "border": "1px solid #ccc",
        "borderRadius": "10px",
        "boxShadow": "2px 2px 5px rgba(0,0,0,0.1)"
    }),

    # -----------------------------
    # TASK 2.5 & 2.6: Graphs
    # -----------------------------
    dcc.Graph(id='recession-graph'),
    dcc.Graph(id='yearly-graph')
])

# -----------------------------
# TASK 2.4: Callback Function
# -----------------------------
@app.callback(
    [Output('output-container', 'children'),
     Output('recession-graph', 'figure'),
     Output('yearly-graph', 'figure')],
    [Input('vehicle-dropdown', 'value'),
     Input('year-dropdown', 'value')]
)
def update_dashboard(vehicle_type, year):
    # Filter data for selected vehicle type
    filtered = df[df['Vehicle_Type'] == vehicle_type]

    # -----------------------------
    # TASK 2.5: Recession Report Graph
    # -----------------------------
    recession_fig = px.line(
        filtered[filtered['Recession'] == True],
        x="Year", y="Sales",
        title=f"{vehicle_type} Sales During Recession",
        markers=True
    )

    # -----------------------------
    # TASK 2.6: Yearly Report Graph
    # -----------------------------
    yearly_data = df[df['Year'] == year]
    yearly_fig = px.bar(
        yearly_data,
        x="Vehicle_Type", y="Sales",
        title=f"Sales by Vehicle Type in {year}",
        color="Vehicle_Type"
    )

    return f"You selected '{vehicle_type}' for year {year}.", recession_fig, yearly_fig

# -----------------------------
# Run App
# -----------------------------
if __name__ == '__main__':
    app.run(debug=True)